In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from functools import wraps
from sklearn.metrics import ndcg_score
from glob import glob
from os import path as osp

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
BASE_DIR = 'race_data/'
USE_COL = ['着順', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差', '単勝', '人気', '馬体重', '調教師', 'horse_id', 'jockey_id']

In [4]:
p_race_csvs = sorted(glob(osp.join(BASE_DIR, 'horse_info_*.csv')))
p_race_csvs

['race_data\\horse_info_202408010101.csv',
 'race_data\\horse_info_202408010102.csv',
 'race_data\\horse_info_202408010103.csv',
 'race_data\\horse_info_202408010104.csv',
 'race_data\\horse_info_202408010106.csv',
 'race_data\\horse_info_202408010107.csv',
 'race_data\\horse_info_202408010108.csv',
 'race_data\\horse_info_202408010109.csv',
 'race_data\\horse_info_202408010110.csv',
 'race_data\\horse_info_202408010111.csv',
 'race_data\\horse_info_202408010112.csv',
 'race_data\\horse_info_202408010201.csv',
 'race_data\\horse_info_202408010202.csv',
 'race_data\\horse_info_202408010205.csv',
 'race_data\\horse_info_202408010206.csv',
 'race_data\\horse_info_202408010207.csv',
 'race_data\\horse_info_202408010208.csv',
 'race_data\\horse_info_202408010209.csv',
 'race_data\\horse_info_202408010210.csv',
 'race_data\\horse_info_202408010211.csv',
 'race_data\\horse_info_202408010212.csv',
 'race_data\\horse_info_202408010301.csv',
 'race_data\\horse_info_202408010302.csv',
 'race_data

In [5]:
def save_dfs_to_excel(file_path, dfs_dict):
    """
    複数のDataFrameを指定されたExcelファイルのシートに保存する関数。

    Parameters:
    - file_path: str
        保存先のExcelファイルパス
    - dfs_dict: dict
        シート名をキー、DataFrameを値とする辞書

    Returns:
    - None
    """
    with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
        for sheet_name, df in dfs_dict.items():
            if isinstance(df, pd.DataFrame):
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            else:
                raise ValueError(f"キー '{sheet_name}' に対応する値がDataFrameではありません。")
    print(f"DataFramesが正常に {file_path} に保存されました。")

In [6]:
def load_processed_df(p_csv, use_col=USE_COL):
    df = pd.read_csv(p_csv)
    df.columns = [c.replace(' ', '') for c in df.columns]
    df = df[use_col]
    df["レースID"] = int(osp.basename(p_csv).split('_')[2].split('.')[0])  # 各DataFrameにレースIDを付与
    df['性'] = df['性齢'].map(lambda x: x[0])
    df['齢'] = df['性齢'].map(lambda x: int(x[1:]))
    df['馬体重'] = df['馬体重'].map(lambda s: s if s != '計不' else None)
    df['当日体重'] = df['馬体重'].map(lambda s: int(s.split('(')[0]) if s is not None else None)
    df['増減'] = df['馬体重'].map(lambda s: int(s.split('(')[1].replace(')', '')) if s is not None else None)
    df.loc[df['着順'] == 1, '着差'] = 0
    df = df.drop(['性齢', '馬体重'], axis=1)
    df = df.loc[~df['着順'].isin(['取', '中', '除', '失'])]
    df['着順'] = (1 + len(df) - df['着順'].astype(int))
    return df

In [7]:
# 複数のDataFrameをリストとして保持
dfs = [load_processed_df(p_race_csv) for p_race_csv in p_race_csvs]

# 単一のデータフレームに統合
merged_df = pd.concat(dfs, ignore_index=True)

merged_df

,着順,馬名,斤量,騎手,タイム,着差,単勝,人気,調教師,horse_id,jockey_id,レースID,性,齢,当日体重,増減
0,16,セレブレイトエール,57.0,坂井瑠星,1:55.4,0,2.6,1.0,[西] 大久保龍,2021104109,1163,202408010101,牡,3,522.0,-4.0
1,15,カフジテルビウム,57.0,長岡禎仁,1:55.9,3,14.7,5.0,[西] 杉山佳明,2021102150,1142,202408010101,牡,3,464.0,4.0
2,14,ルージュシュエット,55.0,鮫島克駿,1:55.9,アタマ,32.3,9.0,[西] 矢作芳人,2021105640,1157,202408010101,牝,3,442.0,-2.0
3,13,ヤマニンアラクリア,57.0,田中健,1:56.2,1.3/4,19.3,7.0,[西] 中村直也,2021100284,1114,202408010101,牡,3,480.0,10.0
4,12,アメリカンチーフ,57.0,松若風馬,1:56.2,クビ,5.0,2.0,[西] 音無秀孝,2021110123,1154,202408010101,牡,3,468.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7489,5,ロゼフレア,53.0,岩田望来,1:22.3,クビ,20.9,7.0,[西] 中村直也,2021103203,1174,202408070812,牝,3,452.0,6.0
7490,4,アサギリ,52.0,永島まな,1:22.3,ハナ,68.7,14.0,[東] 尾形和幸,2021100610,1187,202408070812,牝,3,460.0,-4.0
7491,3,エイムフォーエース,55.0,高杉吏麒,1:22.3,アタマ,21.7,8.0,[東] 武井亮,2021104167,1213,202408070812,牡,3,444.0,0.0
7492,2,クイックバイオ,54.0,佐々木大,1:22.4,3/4,8.4,5.0,[西] 須貝尚介,2021105374,1197,202408070812,牝,3,490.0,-4.0


In [8]:
# レースIDを取得
race_ids = merged_df["レースID"].unique()
train_valid_ids, test_ids = train_test_split(race_ids, test_size=0.2, shuffle=False)
train_ids, valid_ids = train_test_split(train_valid_ids, test_size=0.2, shuffle=False)
print(f'train size: {len(train_ids)}, valid size: {len(valid_ids)}, test size: {len(test_ids)}')

# トレーニングデータとテストデータに分割
train_df = merged_df[merged_df["レースID"].isin(train_ids)]
valid_df = merged_df[merged_df["レースID"].isin(valid_ids)]
test_df = merged_df[merged_df["レースID"].isin(test_ids)]

# グループ情報の再計算
train_group = train_df.groupby("レースID").size().tolist()
valid_group = valid_df.groupby("レースID").size().tolist()
test_group = test_df.groupby("レースID").size().tolist()

# 特徴量とターゲットの抽出
features = ["斤量", "horse_id", "jockey_id", "人気", "当日体重", "増減"]
X_train = train_df[features]
X_valid = valid_df[features]
X_test = test_df[features]
y_train = train_df["着順"]
y_valid = valid_df["着順"]
y_test = test_df["着順"]

train size: 361, valid size: 91, test size: 113


In [9]:
# LightGBM データセットの作成
train_data = lgb.Dataset(X_train, label=y_train, group=train_group)
valid_data = lgb.Dataset(X_valid, label=y_valid, group=valid_group, reference=train_data)
test_data = lgb.Dataset(X_test, label=y_test, group=test_group, reference=train_data)

# モデルのハイパーパラメータ設定
params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "boosting": "gbdt",
    "num_leaves": 31,
    "learning_rate": 0.05,
    "verbose": -1
}

# モデルの学習
model = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    num_boost_round=1000,
)

In [10]:
# テストデータでの予測
y_pred = model.predict(X_test)

# NDCGスコアの計算
true_relevance = [list(y_test)]
predicted_scores = [y_pred]
ndcg = ndcg_score(true_relevance, predicted_scores)
print(f"NDCGスコア: {ndcg}")

NDCGスコア: 0.926851234062691


In [11]:
result_dir = 'outputs/'
result_dfs = dict()
test_df.loc[:, '予測重み'] = y_pred
race_ids = test_df.loc[:, 'レースID'].unique()
for race_id in race_ids:
    result_df = test_df.loc[test_df['レースID'] == race_id, :].copy()
    result_df.loc[:, '予測'] = result_df.loc[:, '予測重み'].rank(ascending=False, method='min')
    test_df.loc[result_df.index, '予測'] = result_df.loc[:, '予測']
    result_df.loc[:, '着順'] = 1 + len(result_df) - result_df.loc[:, '着順']
    result_df = result_df.loc[:, ['レースID', '予測', '着順', '馬名', '人気']].sort_values('予測', ascending=True)
    result_df = result_df.reset_index(drop=True)
    result_dfs[str(result_df['レースID'].values[0])] = result_df
    print(result_df)
    print('---')

           レースID    予測  着順         馬名    人気
0   202408060504   1.0   2  ルージュシークエル   1.0
1   202408060504   2.0   4  ランフォースマイル   3.0
2   202408060504   3.0   1  フォーキャンドルズ   2.0
3   202408060504   4.0   8       ツインギ   6.0
4   202408060504   5.0   9       ディニテ   7.0
5   202408060504   6.0   7    セイフウサツキ   5.0
6   202408060504   7.0  11    ダノンカゼルタ   4.0
7   202408060504   8.0   3      パスコード   8.0
8   202408060504   9.0   6  マーゴットレジーナ   9.0
9   202408060504  10.0   5    ジェットエアー  10.0
10  202408060504  11.0  14   インナーリソース  12.0
11  202408060504  12.0  13  ウインドラブハーツ  13.0
12  202408060504  13.0  12  キッシングセーラー  14.0
13  202408060504  14.0  10    ホリゾンブルー  11.0
---
           レースID    予測  着順         馬名    人気
0   202408060505   1.0   3      シームルグ   2.0
1   202408060505   2.0  13      コンタンゴ   3.0
2   202408060505   3.0   4    サンジャシント   1.0
3   202408060505   4.0   6     アイブリンク   5.0
4   202408060505   5.0   7  ブラックソンソルト   4.0
5   202408060505   6.0   2    デリュージョン   6.0
6   202408060505   7.0  10  

In [12]:
save_dfs_to_excel('outputs/baseline_prediction.xlsx', result_dfs)

DataFramesが正常に outputs/baseline_prediction.xlsx に保存されました。


In [13]:
def calc_mean_rank(test_df, race_id):
    test_df = test_df.copy()
    sub_df = test_df.loc[test_df.loc[:, 'レースID'] == race_id, :]
    sub_df = sub_df.sort_values('予測', ascending=False).reset_index()
    return sub_df.loc[sub_df['着順'] == 1].index.tolist()[0] + 1

mean_rank = np.nanmean([calc_mean_rank(test_df, race_id) for race_id in test_df.loc[:, 'レースID'].unique()])
print(f'平均順位誤差: {mean_rank}')

平均順位誤差: 5.814159292035399
